In [1]:
import matplotlib.pyplot as plt
from itertools import accumulate
import pickle

In [2]:
def diff_predictor(data, dif_list):
    list_pred_rev = []
    len_test = len(dif_list)
    starting_point = data[-len_test -1]
    accum_dif = list(accumulate(dif_list))
    final_pred = [i+starting_point for i in accum_dif]
    return final_pred

In [3]:
def parse_list_from_string(s):
    s = s.strip().lstrip('[').rstrip(']')
    return [float(item.strip()) for item in s.split(',') if item.strip()]

In [4]:
import matplotlib.pyplot as plt
with open('Influenza.pkl', 'rb') as f:
    data_all = pickle.load(f)
data_all

{'0': {'data': [698.0,
   921.0,
   1190.0,
   1736.0,
   2428.0,
   3409.0,
   4750.0,
   6342.0,
   7754.0,
   9212.0,
   6798.0,
   5194.0,
   3985.0,
   2673.0,
   2022.0,
   1542.0,
   971.0,
   565.0,
   397.0,
   340.0,
   201.0,
   100.0,
   91.0,
   84.0,
   71.0,
   50.0,
   44.0,
   47.0,
   45.0,
   60.0,
   81.0,
   56.0,
   57.0,
   96.0,
   118.0,
   105.0,
   139.0,
   180.0,
   185.0,
   216.0,
   200.0,
   263.0,
   287.0,
   338.0,
   432.0,
   547.0,
   634.0,
   874.0,
   1192.0,
   2136.0,
   3619.0,
   6009.0,
   6421.0,
   7224.0,
   8396.0,
   8761.0,
   10456.0,
   12771.0,
   12248.0,
   11226.0,
   7164.0,
   5953.0,
   4743.0,
   4099.0,
   3030.0,
   1838.0,
   1129.0,
   724.0,
   468.0,
   342.0,
   334.0,
   285.0,
   227.0,
   211.0,
   174.0,
   183.0,
   152.0,
   139.0,
   114.0,
   127.0,
   121.0,
   114.0,
   145.0,
   125.0,
   102.0,
   142.0,
   203.0,
   247.0,
   265.0,
   243.0,
   285.0,
   369.0,
   413.0,
   465.0,
   570.0,
   791.0,
  

In [10]:
import openai

def openai_func(step, data_train, latent_num,  latent_train=[], latent_test=[], prompt = prompt):
    
    if latent_num==1:
        str_latent = 'Year'
    elif latent_num==2:
        str_latent = 'Months'
    elif latent_num==3:
        str_latent = 'dates as year-week'
    print(latent_test)
    
    if prompt==1 and latent_train!=[]:
        prompt_1 ={
  "role": "user",
  "content": (
    f"Sequence:\n"
    + "\n".join([f"{latent_train[i]}: {data_train[i]}" for i in range(len(data_train))])
    + "\n" +
    "\n".join([f"{latent}: " for latent in latent_test]) +
    f"\n\nPredict the next {step} values in the sequence.\n"
    f"JUST return the values as a list. No explanation."
  )
}
    elif prompt==2  and latent_train!=[]:
        prompt_1 = {
          "role": "user",
          "content": (
            f"data: {data_train}\n"
            f"{str_latent}: {latent_train}\n"
            f"Prediction {str_latent} in order: {latent_test}\n\n"
            f"Predict the next {step} values of the time series.\n"
            f"JUST return the prediction values as a list. No explanation. Treat values as a sequence."
          )
        }
    elif prompt==3  and latent_train!=[]:
        prompt_1 = {
          "role": "user",
          "content": (
            f"data: {data_train}\n"
            f"{str_latent}: {latent_train}\n"
            f"Prediction {str_latent} in order: {latent_test}\n\n"
            f"The provided sequence represents a univariate time series with aligned covariates ({str_latent}). "
            f"These covariates reflect repeating, cyclical patterns (e.g., days, months, seasons) that often influence the temporal behavior of the main sequence. "
            f"Use both the values and the covariates to infer the underlying pattern.\n\n"
            f"Predict the next {step} values of the time series based on the sequence and the upcoming covariate pattern.\n"
            f"JUST return the prediction values as a list. No explanation. Use your internal logic and treat this as a structured sequential pattern task."
          )
        }
    elif prompt==4 and latent_train!=[]:
        prompt_1 = {
  "role": "user",
  "content": (
    f"From {latent_train[0]} to {latent_train[-1]}, "
    f"there were {data_train} values recorded "
    f"on each week.\n"
    f"Predict JUST the next {step} weeks? JUST return the prediction values as a list of numbers. No explanation."
  )
}
    elif prompt==5 and latent_train!=[]:
        prompt_1 = {
  "role": "user",
  "content": (
    f"You are a helpful assistant that performs time series predictions.\n"
    f"This dataset tracks influenza case counts over time...\n"
    f"Just return the predicted numbers in order. No explanation.\n"
    f"Data: {data_train}\n"
    f"{str_latent}: {latent_train}\n"
    f"Prediction {str_latent} in order: {latent_test}\n\n"
    f"Predict the next {step} values in the sequence.\n"
    f"JUST return the values as a list. Treat values as a time series."
  )
}
    print(prompt_1['content'])
    client = openai.OpenAI(
        api_key="replace with your own..."
    )
    
    
    if latent_train==[]:
        print(f'Entered solo')
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # You can also use "gpt-3.5-turbo" if you want
            messages=[
                {
  "role": "user",
  "content": (
    f"data: {data_train}\n\n"
    f"Predict the next {step} values of the time series.\n"
    f"JUST return the values as a list. No explanation, no code."
  )
}

            ]
        )
    else:
#         print(f'data_train {latent_train}')
#         print(f'latent_test {latent_test}')
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # You can also use "gpt-3.5-turbo" if you want
            messages=[prompt_1]
        )
#     print(response.choices[0].message.content)
#     return response.choices[0].message.content
    print(response.choices[0].message.content)
    return parse_list_from_string(response.choices[0].message.content)


In [15]:
step = 2
level = 0
pred_length = 50
data_dict = data_all
latent_num = 2
prompt = 5


data = data_dict[str(level)]['data']
data_dict[str(level)]['test'] = data[-pred_length:]
data_train = data[:-pred_length]
if latent_num==0:
    latent_data = []
    latent_train = []
    latent_test = []
else:
    latent_data = data_dict[str(level)][f'latent{latent_num}']
    latent_train = latent_data[:-pred_length]
    latent_test = latent_data[-pred_length:]

preds_all = []
diff = False
for i in range(int(pred_length/step)):
    print(f'iteration {i+1} of {int(pred_length/step)}')
#     print()
    start = step * i
    end = min(step * (i + 1), pred_length)
    
    data_train = data[:-pred_length+step*i]
#     print(f'data train is {data_train} with len {len(data_train)}')
#     print(f'latent train is {latent_data[:-pred_length+step*i]} with len {len(latent_data[:-pred_length+step*i])}')
#     print(f'latent test is {latent_test[start:end]} with len {len(latent_test[start:end])}')
    preds = openai_func(step, data_train, latent_num, latent_data[:-pred_length+step*i], latent_test[start:end], prompt)
    print(preds)
    preds_all.extend(preds)



iteration 1 of 25
['January', 'January']
You are a helpful assistant that performs time series predictions.
This dataset tracks influenza case counts over time...
Just return the predicted numbers in order. No explanation.
Data: [698.0, 921.0, 1190.0, 1736.0, 2428.0, 3409.0, 4750.0, 6342.0, 7754.0, 9212.0, 6798.0, 5194.0, 3985.0, 2673.0, 2022.0, 1542.0, 971.0, 565.0, 397.0, 340.0, 201.0, 100.0, 91.0, 84.0, 71.0, 50.0, 44.0, 47.0, 45.0, 60.0, 81.0, 56.0, 57.0, 96.0, 118.0, 105.0, 139.0, 180.0, 185.0, 216.0, 200.0, 263.0, 287.0, 338.0, 432.0, 547.0, 634.0, 874.0, 1192.0, 2136.0, 3619.0, 6009.0, 6421.0, 7224.0, 8396.0, 8761.0, 10456.0, 12771.0, 12248.0, 11226.0, 7164.0, 5953.0, 4743.0, 4099.0, 3030.0, 1838.0, 1129.0, 724.0, 468.0, 342.0, 334.0, 285.0, 227.0, 211.0, 174.0, 183.0, 152.0, 139.0, 114.0, 127.0, 121.0, 114.0, 145.0, 125.0, 102.0, 142.0, 203.0, 247.0, 265.0, 243.0, 285.0, 369.0, 413.0, 465.0, 570.0, 791.0, 1034.0, 1402.0, 1869.0, 2411.0, 3610.0, 6223.0, 11987.0, 15294.0, 16672.0

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error

def analyze_data(level, step, latent, my_dict, preds, plot_fig=True, prompt=prompt):
    data_test = my_dict[str(level)]['test']
    data_pred = preds


    if plot_fig:
        plt.figure(figsize=(10, 5))
        plt.plot(data_test, label='True Data')
        plt.plot(data_pred, label='Predictions')
        plt.legend()
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.title(f'fig_level_{level}_step_{step}_latent_{latent}')
        plt.savefig(f'fig_level_{level}_step_{step}_latent_{latent}_prompt{prompt}')
        plt.show()

    r2 = r2_score(data_test, data_pred)
    mae = mean_absolute_error(data_test, data_pred)
    mape = np.mean(np.abs((np.array(data_test) - np.array(data_pred)) / np.array(data_test))) * 100

    print("\nPerformance Metrics:")
    print(f"{'Metric':<10} | {'Value'}")
    print("-" * 25)
    print(f"{'R^2':<10} | {r2:.4f}")
    print(f"{'MAE':<10} | {mae:.4f}")
    print(f"{'MAPE':<10} | {mape:.2f}%")
    return mape, r2
mape_calc, r_2_calc = analyze_data(level, step, latent_num, data_dict, preds_all, plot_fig=True, prompt=prompt)

In [98]:
pred_dict = {}
pred_dict['data'] = data_dict[str(level)]['data']
pred_dict['pred'] = preds_all
pred_dict['meta'] = [level, step, latent_num, prompt]
pred_dict['MAPE'] = round(mape_calc, 4)
pred_dict['R^2'] = round(r_2_calc, 4)

with open(f'level_{level}_step_{step}_latent_{latent_num}_prompt_{prompt}.pkl', 'wb') as f:
    pickle.dump(pred_dict, f)